# Garmin
The company Garmin headquartered in Schaffhausen, Switzerland, uses different data storage methods than what is standard. Because of this, their data requires some preprocessing to be utilized effectively in data science. This document will handle the Garmin time differences, semicircle conversion, and normalize the speed.

In [7]:
import pandas as pd
import numpy as np
ride = pd.read_csv('assets/bike_data.csv')
ride.head(5)

,Timestamp,Latitude,Longitude,Distance,Altitude,Speed
0,896018545,504719750,-998493490,10.87,285.8,1.773
1,896018560,504717676,-998501870,71.85,285.0,5.533
2,896018566,504716354,-998506792,108.02,284.0,6.485
3,896018575,504714055,-998515244,170.23,284.0,6.951
4,896018584,504711900,-998523278,229.27,285.0,6.224


# Garmin GPS Time
Garmin GPSes don't use standard time; they use what they call "Garmin time," which uses seconds beginning from December 31, 1989. In contrast, standard time uses the number of seconds that have elapsed since midnight on January 1, 1970. We will convert the Garmin time into standard time for the first function, adding `631065600`.

In [8]:
def garmin_time_to_datetime(series):
    '''
        Convert Garmin FIT time by adding the number of seconds from January 1, 1970 to December 31, 1989.
        ----
        series: Takes Garmin time (896018545) and converts to datetime (2018-05-23 14:02:25+00:00)
        return: Standard time
    '''
    return pd.to_datetime(series + 631065600, unit='s', utc=True)

In [9]:
ride['Timestamp_datetime'] = ride.Timestamp.map(garmin_time_to_datetime)
ride.head()

,Timestamp,Latitude,Longitude,Distance,Altitude,Speed,Timestamp_datetime
0,896018545,504719750,-998493490,10.87,285.8,1.773,2018-05-23 14:02:25+00:00
1,896018560,504717676,-998501870,71.85,285.0,5.533,2018-05-23 14:02:40+00:00
2,896018566,504716354,-998506792,108.02,284.0,6.485,2018-05-23 14:02:46+00:00
3,896018575,504714055,-998515244,170.23,284.0,6.951,2018-05-23 14:02:55+00:00
4,896018584,504711900,-998523278,229.27,285.0,6.224,2018-05-23 14:03:04+00:00


# Semicircle Conversion
Additionally, Gramin uses semicircles for locations; these can all be relatively easy to convert into coordinates people can easily understand, like decimal degrees and degrees, minutes, seconds (DMS) like often used in printed maps. For example, 504719750 semicircles would correspond to 42.305121 decimal degrees and 42, 18, 18.43 in DMS.

In [10]:
def semicircles_to_degrees(semicircles):
    '''
    Convert semicircles to decimal degrees
    ----
    semicircles: Series of semicircles
    return: series of decimal degrees
    '''
    return semicircles * (180 / (2**31))

def degrees_to_dms(degrees_fraction):
    ''' 
    Convert degrees to degree, minute, second 3-tuples
    ----
    degrees_fraction: decimal degree series
    return: degree, minute, second tuple
    '''
    degrees = int(degrees_fraction)
    minutes = int(np.round(((degrees_fraction - degrees) * 60), 0))
    seconds = float(np.round((degrees_fraction - degrees - minutes / 60) * 3600, 2))
    return (degrees, abs(minutes), abs(seconds))


def dms_to_degrees(d, m, s):
    ''' 
    Convert degrees, minutes, seconds to decimal degrees
    ----
    d: degree
    m: minutes
    s: seconds
    return: decimal degree series
    '''
    return d+(m/60)+(s/3600)

In [11]:
ride['Latitude_degrees'] = ride['Latitude'].map(semicircles_to_degrees)
ride['Longitude_degrees'] = ride['Longitude'].map(semicircles_to_degrees)
ride['Latitude_dms'] = ride['Latitude_degrees'].map(degrees_to_dms)
ride['Longitude_dms'] = ride['Longitude_degrees'].map(degrees_to_dms)
ride.head()

,Timestamp,Latitude,Longitude,Distance,Altitude,Speed,Timestamp_datetime,Latitude_degrees,Longitude_degrees,Latitude_dms,Longitude_dms
0,896018545,504719750,-998493490,10.87,285.8,1.773,2018-05-23 14:02:25+00:00,42.305121,-83.692758,"(42, 18, 18.43)","(-83, 42, 26.07)"
1,896018560,504717676,-998501870,71.85,285.0,5.533,2018-05-23 14:02:40+00:00,42.304947,-83.693460,"(42, 18, 17.81)","(-83, 42, 23.54)"
2,896018566,504716354,-998506792,108.02,284.0,6.485,2018-05-23 14:02:46+00:00,42.304836,-83.693872,"(42, 18, 17.41)","(-83, 42, 22.06)"
3,896018575,504714055,-998515244,170.23,284.0,6.951,2018-05-23 14:02:55+00:00,42.304643,-83.694581,"(42, 18, 16.72)","(-83, 42, 19.51)"
4,896018584,504711900,-998523278,229.27,285.0,6.224,2018-05-23 14:03:04+00:00,42.304463,-83.695254,"(42, 18, 16.07)","(-83, 42, 17.08)"


# Normalize Speed
For data processing purposes, it is sometimes necessary to normalize speed. This can be done using mean and standard deviation, which can give us an idea of the acceleration. 

In [12]:
def normalize_speed(df, pd_series_name, nsd=2):
    '''
    Take all values that are outside some bound and convert them to the appropriate bound.
    Scale the results to z-scores before returning them
    
    df: DataFrame
    pd_series_name: Name of DataFrame column
    nsd: Standard deviation bounds
    return: normalized speed column in dataframe.
    '''
    pd_series = df[pd_series_name].astype(float)
    
    # Find upper and lower bound for outliers
    avg = np.mean(pd_series)
    sd  = np.std(pd_series)
    
    # Calculate the bounds
    lower_bound = avg - nsd * sd
    upper_bound = avg + nsd * sd
    
    # Collapse in the outliers: replace them with appropriate bound
    df.loc[pd_series < lower_bound , pd_series_name ] = lower_bound
    df.loc[pd_series > upper_bound , pd_series_name ] = upper_bound

    return (df[pd_series_name] - avg) / sd

In [13]:
ride['Speed_normalized'] = normalize_speed(ride, 'Speed')
ride.head()

,Timestamp,Latitude,Longitude,Distance,Altitude,Speed,Timestamp_datetime,Latitude_degrees,Longitude_degrees,Latitude_dms,Longitude_dms,Speed_normalized
0,896018545,504719750,-998493490,10.87,285.8,1.773,2018-05-23 14:02:25+00:00,42.305121,-83.692758,"(42, 18, 18.43)","(-83, 42, 26.07)",-1.773680
1,896018560,504717676,-998501870,71.85,285.0,5.533,2018-05-23 14:02:40+00:00,42.304947,-83.693460,"(42, 18, 17.81)","(-83, 42, 23.54)",-0.114230
2,896018566,504716354,-998506792,108.02,284.0,6.485,2018-05-23 14:02:46+00:00,42.304836,-83.693872,"(42, 18, 17.41)","(-83, 42, 22.06)",0.305929
3,896018575,504714055,-998515244,170.23,284.0,6.951,2018-05-23 14:02:55+00:00,42.304643,-83.694581,"(42, 18, 16.72)","(-83, 42, 19.51)",0.511595
4,896018584,504711900,-998523278,229.27,285.0,6.224,2018-05-23 14:03:04+00:00,42.304463,-83.695254,"(42, 18, 16.07)","(-83, 42, 17.08)",0.190739
